In [ ]:
### ---METHOD HAS BEEN DEPRECATED FROM DATABRICKS---

# Code for mounting the AWS S3 bucket(only needs running once)

#from pyspark.sql.functions import *
#import urllib
## define the credentials path
#delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"
## read the delta table to a Spark DataFrame
#
#aws_keys_df = spark.read.format("delta").load(delta_table_path)
#
## get access key and secret access key from spark dataframe
#ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
#SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
#
## encode the secret access key
#
#ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")
#
## AWS S3 bucket name
#AWS_S3_BUCKET = "user-12885f560a0b-bucket"
## Mount name for the bucket
#MOUNT_NAME = "/mnt/mounted-user-12885f560a0b-bucket"
## Source url
#SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
## Mount the drive
#dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)
#
## Should output True

In [ ]:
#dbutils.fs.unmount("/mnt/mounted-user-12885f560a0b-bucket")

In [ ]:
# File location and type
file_location = "s3a://user-12885f560a0b-bucket/topics/12885f560a0b.pin/partition=0/*.json" 
file_type = "json"  # Corrected file type to match the files being read
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from S3 bucket
pin_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
# Display Spark dataframe to check its content
display(pin_df)

In [ ]:
## Clean pin_df:

# replace empty entries & entries with no relevant data in each column with None:
to_replace_with_none = {
    'description': ['No description available Story format', 'Untitled'],
    'image_src': 'Image src error.',
    'poster_name': 'User Info Error',
    'tag_list': 'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e',
    'title': 'No Title Data Available'
}

cleaned_pin_df = pin_df

for column, values in to_replace_with_none.items():
    if isinstance(values, list):
        for value in values:
            cleaned_pin_df = cleaned_pin_df.replace(value, None, subset=[column])
    else:
        cleaned_pin_df = cleaned_pin_df.replace(values, None, subset=[column])                    
# Using '0' instead of None, as it is a numeric column
cleaned_pin_df = cleaned_pin_df.replace({'User Info Error':'0'}, subset=['follower_count'])


# Perform the necessary transformations on the follower_count to ensure every entry is a number. Make sure the data type for this column is an int

# replace k with 000, M with 000000, and B with 000000000
cleaned_pin_df = cleaned_pin_df.withColumn("follower_count", regexp_replace("follower_count", "k", "000"))
cleaned_pin_df = cleaned_pin_df.withColumn("follower_count", regexp_replace("follower_count", "M", "000000"))
cleaned_pin_df = cleaned_pin_df.withColumn("follower_count", regexp_replace("follower_count", "B", "000000000"))

# convert to int
cleaned_pin_df = cleaned_pin_df.withColumn("follower_count", cleaned_pin_df["follower_count"].cast("int"))

# Ensure that each column containing numeric data has a numeric data type

# --done

# Clean the data in the save_location column to include only the save location path
cleaned_pin_df = cleaned_pin_df.withColumn("save_location", regexp_replace("save_location", "Local save in ", ""))

# Rename the index column to ind.
cleaned_pin_df = cleaned_pin_df.withColumnRenamed("index", "ind")

# Reorder the columns:
# ind, unique_id, title, description, follower_count, poster_name, tag_list, is_image_or_video, image_src, save_location, category
cleaned_pin_df = cleaned_pin_df.select("ind", "unique_id", "title", "description", "follower_count", "poster_name", "tag_list", "is_image_or_video", "image_src", "save_location", "category")
#cleaned_pin_df.printSchema()

display(cleaned_pin_df)

In [ ]:
# File location and type
file_location = "s3a://user-12885f560a0b-bucket/topics/12885f560a0b.geo/partition=0/*.json" 
file_type = "json"  # Corrected file type to match the files being read
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
geo_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)

cleaned_geo_df = geo_df
#To clean the df_geo DataFrame you should perform the following transformations:
#Create a new column coordinates that contains an array based on the latitude and longitude columns
cleaned_geo_df = cleaned_geo_df.withColumn("coordinates", array("latitude", "longitude"))
#Drop the latitude and longitude columns from the DataFrame
cleaned_geo_df = cleaned_geo_df.drop("latitude", "longitude")
#Convert the timestamp column from a string to a timestamp data type
cleaned_geo_df = cleaned_geo_df.withColumn("timestamp", to_timestamp("timestamp"))
#Reorder the DataFrame columns to have the following column order:
#ind
#country
#coordinates
#timestamp
cleaned_geo_df = cleaned_geo_df.select("ind", "country", "coordinates", "timestamp")
# Display Spark dataframe to check its content
#cleaned_geo_df.printSchema()
display(cleaned_geo_df)

In [ ]:
# File location and type
file_location = "s3a://user-12885f560a0b-bucket/topics/12885f560a0b.user/partition=0/*.json" 
file_type = "json"  # Corrected file type to match the files being read
# Ask Spark to infer the schema
infer_schema = "true"
# Read in JSONs from mounted S3 bucket
user_df = spark.read.format(file_type) \
.option("inferSchema", infer_schema) \
.load(file_location)
cleaned_user_df = user_df
#Create a new column user_name that concatenates the information found in the first_name and last_name columns
cleaned_user_df = cleaned_user_df.withColumn("user_name", concat("first_name", lit(" "), "last_name"))
#Drop the first_name and last_name columns from the DataFrame
cleaned_user_df = cleaned_user_df.drop("first_name", "last_name")
#Convert the date_joined column from a string to a timestamp data type
cleaned_user_df = cleaned_user_df.withColumn("date_joined", to_timestamp("date_joined"))
#Reorder the DataFrame columns to have the following column order:
#ind
#user_name
#age
#date_joined
cleaned_user_df = cleaned_user_df.select("ind", "user_name", "age", "date_joined")
# Display Spark dataframe to check its content
display(cleaned_user_df)

In [ ]:
## TASK 4 : 
#Find the most popular Pinterest category people post to based on their country.

#Your query should return a DataFrame that contains the following columns:

#country
#category
#category_count, a new column containing the desired query output

# join the dfs
joined_df = cleaned_geo_df.join(cleaned_pin_df, cleaned_geo_df.ind == cleaned_pin_df.ind, "inner").select("country", "category")

# Grouping by a column and applying an aggregation function
popular_category_df = joined_df.groupBy("country", "category").agg({"category": "count"}).withColumnRenamed("count(category)", "category_count").orderBy("category_count", ascending=False)

display(popular_category_df)

In [ ]:
## TASK 5 : 
# Find which was the most popular category in each year
# Find how many posts each category had between 2018 and 2022.
joined_df = cleaned_geo_df.join(cleaned_pin_df, cleaned_geo_df.ind == cleaned_pin_df.ind, "inner").withColumn("post_year", year("timestamp")).select("timestamp", "category")


# Your query should return a DataFrame that contains the following columns:

# post_year, a new column that contains only the year from the timestamp column
# category
# category_count, a new column containing the desired query output

popular_category_by_year_df = joined_df.withColumn("post_year", year("timestamp")).select("post_year", "category").groupBy("post_year", "category").agg({"category": "count"}).withColumnRenamed("count(category)", "category_count").filter((col("post_year") >= 2018) & (col("post_year") <= 2022)).orderBy("category_count", ascending=False)

display(popular_category_by_year_df)

In [ ]:
## TASK 6 : 
# Find the user with the most followers in each country
#Step 1: For each country find the user with the most followers.
joined_df = cleaned_geo_df.join(cleaned_pin_df, cleaned_geo_df.ind == cleaned_pin_df.ind, "inner").select("country", "poster_name", "follower_count").orderBy("follower_count", ascending=False)

display(joined_df)
#popular_category_df = joined_df.groupBy("country", "category").agg({"category": "count"}).withColumnRenamed("count(category)", "category_count")

#Your query should return a DataFrame that contains the following columns:

#country
#poster_name
#follower_count


In [ ]:
#Step 2: Based on the above query, find the country with the user with most followers.


#Your query should return a DataFrame that contains the following columns:
highest_user = joined_df.agg(max("follower_count").alias("highest_user")).collect()[0]["highest_user"]

result = joined_df.filter(col("follower_count") == highest_user).select("country", "follower_count")
#country
#follower_count
#This DataFrame should have only one entry.
display(result)

In [ ]:
## TASK 7 : 
# Find the most popular category for different age groups
# What is the most popular category people post to based on the following age groups:

#window_spec = Window.partitionBy("age").orderBy()
joined_df = cleaned_user_df.join(cleaned_pin_df, cleaned_user_df.ind == cleaned_pin_df.ind, "inner").select("age", "category")

temp_df = joined_df.withColumn(
    "age_group",
    when(col("age") < 18, "Unspecified")
    .when((col("age") >= 18) & (col("age") < 25), "18-24")
    .when((col("age") >= 25) & (col("age") < 36), "25-35")
    .when((col("age") >= 36) & (col("age") < 50), "36-50")
    .otherwise("+50")
)

result = temp_df.select("age_group", "category").groupBy("age_group", "category").agg({"category": "count"}).withColumnRenamed("count(category)", "category_count").orderBy("category_count", ascending=False)
# 18-24
# 25-35
# 36-50
# +50
# Your query should return a DataFrame that contains the following columns:

# age_group, a new column based on the original age column
# category
# category_count, a new column containing the desired query output
display(result)

In [ ]:
## TASK 8 : 
# Find the median follower count for the different age groups
# What is the median follower count for users in the following age groups:

# 18-24
# 25-35
# 36-50
# +50
#Your query should return a DataFrame that contains the following columns:

#age_group, a new column based on the original age column
#median_follower_count, a new column containing the desired query output
from pyspark.sql import functions as F

joined_df = cleaned_user_df.join(cleaned_pin_df, cleaned_user_df.ind == cleaned_pin_df.ind, "inner").select("age", "follower_count").withColumn(
    "age_group",
    when(col("age") < 18, "Unspecified")
    .when((col("age") >= 18) & (col("age") < 25), "18-24")
    .when((col("age") >= 25) & (col("age") < 36), "25-35")
    .when((col("age") >= 36) & (col("age") < 50), "36-50")
    .otherwise("+50")
)
temp_df = joined_df.select("age_group", "follower_count")
agg_df = joined_df.groupBy("age_group") \
    .agg(
        F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count")
    ).orderBy("median_follower_count", ascending=False)

display(agg_df)

In [ ]:
## TASK 9 : 
# Find out how many users have joined each year
#Find how many users have joined between 2015 and 2020.


#Your query should return a DataFrame that contains the following columns:


#post_year, a new column that contains only the year from the timestamp column
#number_users_joined, a new column containing the desired query output

joined_df = cleaned_user_df.join(cleaned_geo_df, cleaned_user_df.ind == cleaned_geo_df.ind, "inner").select("timestamp", "date_joined").withColumn("post_year", year("timestamp")).withColumn("join_year",year("date_joined"))

grouped_df = joined_df.groupBy("join_year").agg(count("join_year").alias("number_users_joined")).filter((col("join_year") >= 2015) & (col("join_year") <= 2020)).orderBy("join_year", ascending=True)
display(grouped_df)

In [ ]:
## TASK 10 : 
# Find the median follower count of users have joined between 2015 and 2020.


# Your query should return a DataFrame that contains the following columns:

# post_year, a new column that contains only the year from the timestamp column
# median_follower_count, a new column containing the desired query output
joined_df = cleaned_user_df.join(cleaned_pin_df, cleaned_user_df.ind == cleaned_pin_df.ind, "inner").select("date_joined","follower_count").withColumn("join_year",year("date_joined"))
temp_df = joined_df.select("join_year", "follower_count")
result = temp_df.groupBy("join_year").agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count")).filter((col("join_year") >= 2015) & (col("join_year") <= 2020)).orderBy("join_year", ascending=True)

display(result)

In [ ]:
## TASK 11 : 
# Find the median follower count of users based on their joining year and age group
#Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.


#Your query should return a DataFrame that contains the following columns:

#age_group, a new column based on the original age column
#post_year, a new column that contains only the year from the timestamp column
#median_follower_count, a new column containing the desired query output

joined_df = cleaned_user_df.join(cleaned_pin_df, cleaned_user_df.ind == cleaned_pin_df.ind, "inner").select("age", "follower_count", "date_joined")

temp_df = joined_df.withColumn(
    "age_group",
    when(col("age") < 18, "Unspecified")
    .when((col("age") >= 18) & (col("age") < 25), "18-24")
    .when((col("age") >= 25) & (col("age") < 36), "25-35")
    .when((col("age") >= 36) & (col("age") < 50), "36-50")
    .otherwise("+50")
).withColumn("join_year",year("date_joined")).select("age_group", "join_year", "follower_count").filter("join_year >= 2015 and join_year <= 2020")
result = temp_df.groupBy("age_group", "join_year").agg(F.expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))
display(result)